<a href="https://colab.research.google.com/github/lokesh1435/Clinical-Data-Analysis/blob/main/Lokesh_Kumar_Patnaik_Clinical_Report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas scispacy spacy
!pip install en_core_sci_md


  Using cached spacy-3.7.6-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (27 kB)
  Using cached thinc-8.2.5-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (15 kB)
Using cached spacy-3.7.6-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (29.0 MB)
Using cached thinc-8.2.5-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (922 kB)
  Attempting uninstall: thinc
    Found existing installation: thinc 8.1.12
    Uninstalling thinc-8.1.12:
      Successfully uninstalled thinc-8.1.12
  Attempting uninstall: spacy
    Found existing installation: spacy 3.4.4
    Uninstalling spacy-3.4.4:
      Successfully uninstalled spacy-3.4.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
en-core-sci-md 0.5.1 requires spacy<3.5.0,>=3.4.1, but you have spacy 3.7.6 which is incompatible.
en-core-web-md 3.4.1 requires spac

In [ ]:
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.1/en_core_med7_lg-0.5.1.tar.gz
!pip install pandas spacy scikit-learn
!python -m spacy download en_core_web_md

  ERROR: HTTP error 404 while getting https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.1/en_core_med7_lg-0.5.1.tar.gz
ERROR: Could not install requirement https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.1/en_core_med7_lg-0.5.1.tar.gz because of HTTP error 404 Client Error: Not Found for url: https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.1/en_core_med7_lg-0.5.1.tar.gz for URL https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.1/en_core_med7_lg-0.5.1.tar.gz
2024-08-25 21:24:03.697144: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-25 21:24:03.819136: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-25 21:24:03.845025: E external/local_xla/xla/

In [ ]:
import pandas as pd
import spacy
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

# Load spaCy model
nlp = spacy.load('en_core_web_md')  # Use a commonly available spaCy model

class ClinicalTextAnalysis:
    def __init__(self, path='healthcare_dataset.csv'):
        self.path = path
        self.data = pd.read_csv(self.path)
        self.preprocessed_data = None

    def preprocess_text(self, text):
        # Use spaCy for entity extraction
        doc = nlp(text)
        entities = [ent.text for ent in doc.ents]
        return ' '.join(entities)

    def preprocess_data(self):
        # Apply preprocessing to the relevant columns
        self.data['Processed Medical Condition'] = self.data['Medical Condition'].apply(self.preprocess_text)
        self.data['Processed Test Results'] = self.data['Test Results'].apply(self.preprocess_text)
        self.preprocessed_data = self.data[['Processed Medical Condition', 'Processed Test Results']]

    def extract_topics(self, num_topics=5):
        # Combine text columns for topic modeling
        combined_text = self.preprocessed_data['Processed Medical Condition'] + ' ' + self.preprocessed_data['Processed Test Results']

        # Vectorize text and perform topic modeling
        vectorizer = CountVectorizer(stop_words='english')
        lda = LatentDirichletAllocation(n_components=num_topics, random_state=0)
        pipeline = make_pipeline(vectorizer, lda)

        # Fit the model and display topics
        pipeline.fit(combined_text)
        feature_names = vectorizer.get_feature_names_out()
        for topic_idx, topic in enumerate(lda.components_):
            print(f"Topic #{topic_idx}:")
            print(" ".join([feature_names[i] for i in topic.argsort()[:-10 - 1:-1]]))

    def analyze_data(self):
        # Print data and preprocess information
        print("Data Head:\n", self.data.head())
        print("Preprocessed Data Head:\n", self.preprocessed_data.head())

        # Example of scaling numerical data
        if 'Billing Amount' in self.data.columns:
            scaler = StandardScaler()
            self.data['Billing Amount (scaled)'] = scaler.fit_transform(self.data[['Billing Amount']])
            print("Scaled Billing Amount:\n", self.data[['Billing Amount', 'Billing Amount (scaled)']].head())

    def get_summary(self):
        # Execute the main steps
        self.preprocess_data()
        self.extract_topics()
        self.analyze_data()

if __name__ == "__main__":
    analysis = ClinicalTextAnalysis()
    analysis.get_summary()


Topic #0:
cancer
Topic #1:
cancer
Topic #2:
cancer
Topic #3:
cancer
Topic #4:
cancer
Data Head:
             Name  Age  Gender Blood Type Medical Condition Date of Admission  \
0  Bobby JacksOn   30    Male         B-            Cancer        2024-01-31   
1   LesLie TErRy   62    Male         A+           Obesity        2019-08-20   
2    DaNnY sMitH   76  Female         A-           Obesity        2022-09-22   
3   andrEw waTtS   28  Female         O+          Diabetes        2020-11-18   
4  adrIENNE bEll   43  Female        AB+            Cancer        2022-09-19   

             Doctor                    Hospital Insurance Provider  \
0     Matthew Smith             Sons and Miller         Blue Cross   
1   Samantha Davies                     Kim Inc           Medicare   
2  Tiffany Mitchell                    Cook PLC              Aetna   
3       Kevin Wells  Hernandez Rogers and Vang,           Medicare   
4    Kathleen Hanna                 White-White              Aetna   

 